# LogisticRegression

## Import and Load Data Set

In [1]:
import pandas as pd
import numpy
import seaborn
from datetime import datetime
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from verstack.stratified_continuous_split import scsplit 

In [2]:
# Load the training data
train_data = pd.read_csv("data/train.csv")
train_data.head(3)

,id,timestamp,retweet_count,user_verified,user_statuses_count,user_followers_count,user_friends_count,user_mentions,urls,hashtags,text
0,0,1588696955143,0,False,68460,1101,1226,NaN,NaN,NaN,Smh I give up
1,1,1588464948124,0,False,309,51,202,NaN,NaN,NaN,"Most of us are Human Beings, but I think you m..."
2,2,1588634673360,0,False,3241,1675,2325,NaN,NaN,NaN,"Old dirty tricks Trump, at it again...like we ..."


In [3]:
train_data['retweet_count'].describe()

count    665777.000000
mean        147.687398
std        2972.051181
min           0.000000
25%           0.000000
50%           0.000000
75%           2.000000
max      942572.000000
Name: retweet_count, dtype: float64

In [4]:
train_data['retweet_count'].value_counts()

0       422803
1        57111
2        25737
3        16338
4        11695
         ...  
2279         1
2468         1
4324         1
1765         1
5301         1
Name: retweet_count, Length: 6871, dtype: int64

## Preprocessing

In [5]:
class FeatureExtraction(object):

    def __init__(self, train_data):
        self.train_data = train_data
    
    def transform(self):
        train_data = self.train_data
        
        #user verified
        labelencoder=LabelEncoder()
        train_data['user_verified']=labelencoder.fit_transform(train_data['user_verified'])
        
        #timestamp
        train_data["user_hour"] = train_data.timestamp.apply(lambda t: (datetime.fromtimestamp(t//1000-5*3600)).hour)
        train_data["user_day"] = train_data.timestamp.apply(lambda t: (datetime.fromtimestamp(t//1000-5*3600)).weekday())
        train_data["user_month"] = train_data.timestamp.apply(lambda t: (datetime.fromtimestamp(t//1000-5*3600)).month)
        
        #number of hashtags
        train_data['totalhashtag'] = train_data['hashtags'].str.split().str.len()
        train_data['totalhashtag'] = train_data['totalhashtag'].fillna(0)
        train_data = train_data.drop(columns=['hashtags'])
        

        #number of mentions
        train_data['totalmentions'] = train_data['user_mentions'].str.split().str.len()
        train_data['totalmentions'] = train_data['totalmentions'].fillna(0)
        
        train_data = train_data.drop(columns=['user_mentions'])
        
        
        # urls
        train_data['urls'] = train_data['urls'].fillna(0)
        train_data = train_data.assign(urls=(train_data['urls'] !=0 ).astype(int))
        
      
        self.transformed_data = train_data
                                     
        pass

## Feature Extraction


In [6]:
y = train_data.retweet_count 
for i in range(train_data.shape[0]):
       if y[i] == 0:
            y[i] = 0
       elif y[i]>0 and y[i]<10:
            y[i] = 1
       elif y[i]>10 and y[i]<100:
            y[i] = 2
       elif y[i]>100 and y[i]<1000:
            y[i] = 3
       else:
            y[i] = 4

In [7]:
FE = FeatureExtraction(train_data)
FE.transform()
features = FE.transformed_data

In [8]:
X = features[["user_verified", "user_statuses_count", "user_followers_count", "user_friends_count","totalhashtag", "totalmentions","urls","user_hour","user_day","user_month"]]
X_train, X_test, y_train, y_test = scsplit(X, y, stratify=y, train_size=0.7,test_size=0.3)

## Logistic Regression

In [9]:
train_data_model = LogisticRegression()
train_data_model.fit(X_train,y_train)
y_pred=train_data_model.predict(X_test)

In [10]:
metrics.mean_absolute_error(y_test, y_pred)

0.4310883475021779

In [11]:
acc = accuracy_score(y_test,y_pred)
acc

0.6822724223216878

In [12]:
y.describe()

count    665777.000000
mean          0.598619
std           0.956913
min           0.000000
25%           0.000000
50%           0.000000
75%           1.000000
max           4.000000
Name: retweet_count, dtype: float64

## Evaluation 

In [13]:
eval_data = pd.read_csv("data/evaluation.csv")
FE_eval = FeatureExtraction(eval_data)
FE_eval.transform()

In [14]:
trans_eval_data = FE_eval.transformed_data
X_eval = trans_eval_data.drop("text",axis=1)

In [15]:
sc = StandardScaler()
X_eval = sc.fit_transform(X_eval.values)

## Prediction for Logistic Regression

In [16]:
y_eval_pred_logistic = train_data_model.predict(X_eval)

ValueError: X has 12 features per sample; expecting 10

In [ ]:
with open("logistic_regression.txt", 'w') as f:
    writer = csv.writer(f)
    writer.writerow(["TweetID", "NoRetweets"])
    for index, prediction in enumerate(y_eval_pred_logistic):
        writer.writerow([str(trans_eval_data['id'].iloc[index]) , str(int(prediction))])